In [4]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import pandas as pd

#!pip install wikipedia
import wikipedia as wp

#!pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!pip install folium
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium


url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
#print(soup.prettify())
print("done")
right_table=soup.find('table', class_='wikitable sortable')
#right_table
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

converted_postalcode = []
converted_Borough = []
converted_Neighborhood = []

for element in df['Postal Code']:
    converted_postalcode.append(element.strip())
for element in df['Borough']:
    converted_Borough.append(element.strip())
for element in df['Neighborhood']:
    converted_Neighborhood.append(element.strip())
    
#print(converted_list)
df['Postal Code']=converted_postalcode
df['Borough']=converted_Borough
df['Neighborhood']=converted_Neighborhood
#df.head()
url2 = "https://cocl.us/Geospatial_data"
df2 = pd.read_csv(url2)
Torontolist = pd.merge(left=df, right=df2, left_on='Postal Code', right_on='Postal Code')
Torontolist.shape
Torontolist.head()

done


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [5]:
Torontolist['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

# Toronto has 5 boroughs(Districts) and 306 neighborhoods, these are represented in blue circles in the following map

In [6]:
import folium

address3 = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="cn_explorer")
location = geolocator.geocode(address3)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude1, longitude1))

toronto_map = folium.Map(location=[latitude1, longitude1], zoom_start=11)
for lat1, lng1, borough, neighborhood in zip(Torontolist['Latitude'], Torontolist['Longitude'], Torontolist['Borough'], Torontolist['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat1, lng1],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


# Code to extract information for paris city into a data frame
## Information is taken from 2 different wiki pages and latitude and longitude information has been manually added for 10 neighborhoods from google since they were un available in foursquare database

In [7]:
%%capture
import pandas as pd
import wikipedia as wp
 
#Get the html source
html = wp.page("Quarters of Paris").html().encode("UTF-8")
df6 = pd.read_html(html)[0]
df6.to_csv('beautifulsoup_pandas.csv',header=0,index=False)
#df6.head()
Columns = ['Arrondissement(Districts)', 'Quartiers(Quarters).1']
df6=df6[Columns]
df6.head()

df6['Arrondissement(Districts)'] = df6['Arrondissement(Districts)'].str.replace('arrondissement', ',')
df6['Arrondissement(Districts)'] = df6['Arrondissement(Districts)'].str.replace('Called ', '')
df6['Arrondissement(Districts)'] = df6['Arrondissement(Districts)'].str.replace('"', '')
df6['Arrondissement(Districts)'] = df6['Arrondissement(Districts)'].str.replace('(', '')
df6['Arrondissement(Districts)'] = df6['Arrondissement(Districts)'].str.replace(')', '')
df6['Arrondissement(Districts)'] = df6['Arrondissement(Districts)'].str.replace('du', '')
df6['Arrondissement(Districts)'] = df6['Arrondissement(Districts)'].str.replace('de', '')
df6['Arrondissement(Districts)'] = df6['Arrondissement(Districts)'].str.replace('des', '')

#df6['Arrondissement(Districts)'] = df6['Arrondissement(Districts)'].str.replace(',(', '')

df6.columns =['Arrondissement(District)', 'Quartiers(Neighbourhood)']
#df6.head()
import numpy as np
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

lat=[]
long=[]

for i in df6['Quartiers(Neighbourhood)']:
    address = i
    #print(address)
    geolocator = Nominatim(user_agent="eu_agent")
    location = geolocator.geocode(address)
    try:
        latitude = location.latitude
        lat.append(latitude)
    except AttributeError:
        latitude = np.nan
        lat.append(latitude)
    try:
        longitude = location.longitude
        long.append(longitude)
    except AttributeError:
        longitude = np.nan
        long.append(longitude)
    #print(latitude, longitude)
    
df6['latittude']=lat
df6['longitude']=long

#df6.head()
df6["latittude"][3] = 46.782050
df6["longitude"][3] = -0.835640
df6["latittude"][8] = 45.072760
df6["longitude"][8] = -1.145780
df6["latittude"][18] = 48.780580
df6["longitude"][18] = -0.511530
df6["latittude"][21] = 8.667050
df6["longitude"][21] = 41.477050
df6["latittude"][23] = 48.858125
df6["longitude"][23] = 2.333992
df6["latittude"][26] = 48.849567
df6["longitude"][26] = 2.314755
df6["latittude"][28] = 48.868138
df6["longitude"][28] = 2.306459
df6["latittude"][33] = 48.874802
df6["longitude"][33] = 2.332949
df6["latittude"][39] = 48.876187
df6["longitude"][39] = 2.366164
df6["latittude"][40] = 48.868187
df6["longitude"][40] = 2.373675
df6["latittude"][48] = 48.838705
df6["longitude"][48] = 2.366831
df6["latittude"][67] = 48.893826
df6["longitude"][67] = 2.322759
df6["latittude"][68] = 48.891517
df6["longitude"][68] = 2.330561
df6["latittude"][74] = 48.882613
df6["longitude"][74] = 2.393059
df6["latittude"][78] = 48.864554
df6["longitude"][78] = 2.395221
;

In [8]:
df6.head()

,Arrondissement(District),Quartiers(Neighbourhood),latittude,longitude
0,"1st , Louvre",Saint-Germain-l'Auxerrois,48.860211,2.336299
1,"1st , Louvre",Les Halles,45.719108,4.431157
2,"1st , Louvre",Palais-Royal,48.863585,2.336204
3,"1st , Louvre",Place-VendÃ´me,46.782050,-0.835640
4,"2nd , la Bourse",Gaillon,49.160560,1.333613


# Paris has 20 Arrondissements(Districts). Each Arrondissement has 4 Neighbourhoods called Quartiers with total of 80 Quartiers. These are represented in following map

In [9]:
address4 = 'Paris, France'

geolocator = Nominatim(user_agent="eu_explorer")
location = geolocator.geocode(address4)
latitude2 = location.latitude
longitude2 = location.longitude
print('The geograpical coordinate of Paris City are {}, {}.'.format(latitude2, longitude2))

paris_map = folium.Map(location=[latitude2, longitude2], zoom_start=12)

for lat2, lng2, Arrondissement, Quartiers in zip(df6['latittude'], df6['longitude'], df6['Arrondissement(District)'], df6['Quartiers(Neighbourhood)']):
    label = '{}, {}'.format(Quartiers, Arrondissement)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat2, lng2],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(paris_map)  
    
paris_map

The geograpical coordinate of Paris City are 48.8566969, 2.3514616.


In [23]:
CLIENT_ID = 'GZMTIXZAYGD5WCKI3ASB3KZJ0VEUBU5W2VFUMEHPMR20UADD' 
CLIENT_SECRET = 'HV3ORFZW4WIQY04BWI0KIH3TF5X1RNGXEJM0AGATIS5W2XPI' 
#version: YYYYMMDD format
VERSION = '20200516'
LIMIT = 30

In [24]:
Torontolist.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [25]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [26]:
%%capture
Toronto_venues = getNearbyVenues(names=Torontolist['Neighborhood'],
                                   latitudes=Torontolist['Latitude'],
                                   longitudes=Torontolist['Longitude']
                                  )

In [27]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1325, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [28]:
print('There are {} uniques categories of venues in Toronto.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 229 uniques categories of venues in Toronto.


In [29]:
# one hot encoding
Toronto_venues2 = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_venues2['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_venues2.columns[-1]] + list(Toronto_venues2.columns[:-1])
Toronto_venues2 = Toronto_venues2[fixed_columns]

Toronto_grouped = Toronto_venues2.groupby('Neighborhood').mean().reset_index()
#Toronto_grouped.head()

# Here we have data for top 5 venues for each of the neighbourhood

In [30]:
num_top_venues=5
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Clothing Store,Skating Rink,Lounge
1,"Alderwood, Long Branch",Pizza Place,Skating Rink,Pharmacy,Coffee Shop,Pool
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Deli / Bodega,Ice Cream Shop
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dim Sum Restaurant
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Hobby Shop,Café


In [31]:
neighborhoods_venues_sorted['1st Most Common Venue'].value_counts().nlargest(5)

Café              12
Park              10
Pizza Place        8
Coffee Shop        8
Sandwich Place     4
Name: 1st Most Common Venue, dtype: int64

In [32]:
print("top 5 2nd most common venues")
neighborhoods_venues_sorted['2nd Most Common Venue'].value_counts().nlargest(5)

top 5 2nd most common venues


Coffee Shop      13
Women's Store     5
Café              5
Park              5
Pub               3
Name: 2nd Most Common Venue, dtype: int64

In [33]:
print("top 5 3rd most common venues")
neighborhoods_venues_sorted['3rd Most Common Venue'].value_counts().nlargest(5)

top 5 3rd most common venues


Coffee Shop            7
Women's Store          6
Bakery                 4
Gym                    4
Japanese Restaurant    4
Name: 3rd Most Common Venue, dtype: int64

In [34]:
print("top 5 4th most common venues")
neighborhoods_venues_sorted['4th Most Common Venue'].value_counts().nlargest(5)

top 5 4th most common venues


Department Store    7
Coffee Shop         4
Women's Store       4
Bank                4
Café                3
Name: 4th Most Common Venue, dtype: int64

In [35]:
print("top 5 5th most common venues")
neighborhoods_venues_sorted['5th Most Common Venue'].value_counts().nlargest(5)

top 5 5th most common venues


Event Space           8
Women's Store         8
Dessert Shop          4
Falafel Restaurant    4
Department Store      3
Name: 5th Most Common Venue, dtype: int64

# Overall, most common venues of toronto are : Coffee Shops/Café , followed by parks, bakeries

# Clustering toronto neighbourhoods

In [36]:
from sklearn.cluster import KMeans

kclusters = 15

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15]

array([12,  1,  8, 12,  8,  8,  8,  8,  8,  8, 10,  8, 12,  8,  8],
      dtype=int32)

In [45]:
#neighborhoods_venues_sorted.drop(columns=['Cluster Labels'])

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Torontolist

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
#toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].fillna(toronto_merged["Cluster Labels"].median())
#toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)
toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,6.0,Park,Construction & Landscaping,Food & Drink Shop,Women's Store,Dessert Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Deli / Bodega
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,8.0,Coffee Shop,Park,Bakery,Theater,Breakfast Spot
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,8.0,Clothing Store,Accessories Store,Boutique,Event Space,Vietnamese Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,8.0,Coffee Shop,Sushi Restaurant,Gym,Fried Chicken Joint,Sandwich Place


In [46]:
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].fillna(toronto_merged["Cluster Labels"].median())
toronto_merged["Cluster Labels"] = toronto_merged["Cluster Labels"].astype(int)

In [47]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
toronto_clusters =  toronto_map
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    #print(cluster)
    label = folium.Popup(str(poi) + ' cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=7,
        popup=label,
        color=rainbow[cluster-2],
        fill=True,
        fill_color=rainbow[cluster-2],
        fill_opacity=0.7).add_to(toronto_clusters)
       
toronto_clusters


In [48]:
toronto_merged['Cluster Labels'].value_counts()

8     58
12    12
1     10
4      6
6      4
10     3
0      2
14     1
13     1
11     1
9      1
7      1
5      1
3      1
2      1
Name: Cluster Labels, dtype: int64

# We can see that one neighbourhoods are most closely placed towards the lake shore where we have Downtown Core (Central) and are more scattered farther away from the shore


# Similar analysis for the city of Paris. Top 5 venues of each neighbourhood of paris

In [49]:
parislist = df6
parislist.head()

,Arrondissement(District),Quartiers(Neighbourhood),latittude,longitude
0,"1st , Louvre",Saint-Germain-l'Auxerrois,48.860211,2.336299
1,"1st , Louvre",Les Halles,45.719108,4.431157
2,"1st , Louvre",Palais-Royal,48.863585,2.336204
3,"1st , Louvre",Place-VendÃ´me,46.782050,-0.835640
4,"2nd , la Bourse",Gaillon,49.160560,1.333613


In [53]:
%%capture
address4 = 'Paris, France'

geolocator = Nominatim(user_agent="cn_explorer")
location = geolocator.geocode(address4)
latitude2 = location.latitude
longitude2 = location.longitude


In [54]:
%%capture
paris_venues = getNearbyVenues(names=parislist['Quartiers(Neighbourhood)'],
                                   latitudes=parislist['latittude'],
                                   longitudes=parislist['longitude']
                                  )

In [56]:
print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 231 uniques categories.


In [57]:
paris_venues2 = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_venues2['Neighborhood'] = paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns2 = [paris_venues2.columns[-1]] + list(paris_venues2.columns[:-1])
paris_venues2 = paris_venues2[fixed_columns2]


paris_grouped = paris_venues2.groupby('Neighborhood').mean().reset_index()


In [58]:
num_top_venues=5
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted2 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted2['Neighborhood'] = paris_grouped['Neighborhood']

for ind in np.arange(paris_grouped.shape[0]):
    neighborhoods_venues_sorted2.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted2.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,AmÃ©rique,French Restaurant,Arts & Crafts Store,Supermarket,Theater,Bistro
1,Archives,Clothing Store,Cocktail Bar,Sandwich Place,Deli / Bodega,Art Gallery
2,Arsenal,Soccer Stadium,Park,Pub,Gym / Fitness Center,Sports Bar
3,Arts-et-MÃ©tiers,Ice Cream Shop,Cocktail Bar,Resort,Bar,Donut Shop
4,Batignolles,French Restaurant,Wine Bar,Coffee Shop,Italian Restaurant,Bar


In [59]:
print("top 5 most common venues")
neighborhoods_venues_sorted2['1st Most Common Venue'].value_counts().nlargest(5)

top 5 most common venues


French Restaurant    26
Hotel                11
Bar                   5
Plaza                 2
Lake                  1
Name: 1st Most Common Venue, dtype: int64

In [60]:
print("top 5 2nd most common venues")
neighborhoods_venues_sorted2['2nd Most Common Venue'].value_counts().nlargest(5)

top 5 2nd most common venues


French Restaurant     11
Hotel                  9
Park                   3
Italian Restaurant     3
Wine Bar               3
Name: 2nd Most Common Venue, dtype: int64

In [61]:
print("top 5 3rd most common venues")
neighborhoods_venues_sorted2['3rd Most Common Venue'].value_counts().nlargest(5)

top 5 3rd most common venues


Hotel                 8
Zoo Exhibit           6
Italian Restaurant    4
Pizza Place           3
Sandwich Place        3
Name: 3rd Most Common Venue, dtype: int64

In [62]:
print("top 5 4th most common venues")
neighborhoods_venues_sorted2['4th Most Common Venue'].value_counts().nlargest(5)

top 5 4th most common venues


Italian Restaurant     7
Historic Site          4
Cocktail Bar           3
Japanese Restaurant    3
Doner Restaurant       3
Name: 4th Most Common Venue, dtype: int64

In [63]:
print("top 5 5th most common venues")
neighborhoods_venues_sorted2['5th Most Common Venue'].value_counts().nlargest(5)

top 5 5th most common venues


Fast Food Restaurant    5
French Restaurant       4
Japanese Restaurant     4
Bistro                  4
Bar                     3
Name: 5th Most Common Venue, dtype: int64

# Top most common venues in paris are French Restaurants, Italian Restaurents, Hotels and Historic sites

# Clustering of Paris neighbourhoods

In [64]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 15

paris_grouped_clustering = paris_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15] 


array([6, 1, 1, 5, 6, 3, 9, 6, 8, 6, 8, 5, 8, 5, 5], dtype=int32)

In [67]:
neighborhoods_venues_sorted2.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = parislist

paris_merged = paris_merged.join(neighborhoods_venues_sorted2.set_index('Neighborhood'), on='Quartiers(Neighbourhood)')

paris_merged['Cluster Labels'].fillna(paris_merged['Cluster Labels'].mean(), inplace=True)

paris_merged['Cluster Labels']=paris_merged['Cluster Labels'].apply(np.int64)
paris_merged.head()

,Arrondissement(District),Quartiers(Neighbourhood),latittude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"1st , Louvre",Saint-Germain-l'Auxerrois,48.860211,2.336299,1,Plaza,Art Museum,Hotel,Cosmetics Shop,Historic Site
1,"1st , Louvre",Les Halles,45.719108,4.431157,10,French Restaurant,Clothing Store,Zoo Exhibit,Doner Restaurant,Fast Food Restaurant
2,"1st , Louvre",Palais-Royal,48.863585,2.336204,1,Plaza,Udon Restaurant,Hotel,Historic Site,Japanese Restaurant
3,"1st , Louvre",Place-VendÃ´me,46.782050,-0.835640,11,Hostel,Gastropub,Zoo Exhibit,Doner Restaurant,Fast Food Restaurant
4,"2nd , la Bourse",Gaillon,49.160560,1.333613,1,Flower Shop,Park,Bank,Castle,Financial or Legal Service


In [68]:
#paris_map

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['latittude'], paris_merged['longitude'], paris_merged['Quartiers(Neighbourhood)'], paris_merged['Cluster Labels']):
    #print(cluster)
    label = folium.Popup(str(poi) + ' cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=4,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(paris_map)
       
paris_map


In [69]:
paris_merged['Cluster Labels'].value_counts()

5     24
6     21
1     16
8      8
14     1
13     1
12     1
11     1
10     1
9      1
7      1
4      1
3      1
2      1
0      1
Name: Cluster Labels, dtype: int64

# Here we can see that unlike toronto city, In paris neighborhoods spread out from center of the city

# Further analysis and findings have been provided in the report